In [1]:
import mlflow

In [2]:
experiment_name = "evaluation"

In [3]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [8]:
X, y = make_classification(n_samples=1000, n_features=4, random_state = 3, scale = 10, n_informative = 3, n_redundant = 1)
X = pd.DataFrame(X, columns=[f"f_{i}" for i in range(X.shape[1])])
y = pd.Series(y, name="target")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)
feature_names = X.columns.tolist()

eval_data = X_test.copy()
eval_data["target"] = y_test

In [ ]:
rfc = RandomForestClassifier()

rfc.fit(X_train, y_train)

artifact_path = "random-forest-classifier"

with mlflow.start_run(run_name = "evaluating_rfc") as run:
    mlflow.sklearn.log_model(rfc, artifact_path, registered_model_name = "random-forest-classifier")


    result = mlflow.evaluate(
        model = f"runs:/{run.info.run_id}/{artifact_path}",
        data = eval_data,
        model_type = "classifier",
        targets = "target",
        feature_names = feature_names,
    )


2025/09/13 18:37:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'random-forest-classifier' already exists. Creating a new version of this model...
Created version '2' of model 'random-forest-classifier'.
c:\Users\saqli\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlf

In [12]:
eval_data.head()

,f_0,f_1,f_2,f_3,target
642,13.390612,16.589707,-22.432819,13.999437,1
762,13.403873,-0.407266,14.633971,-3.268620,0
909,16.805758,13.792503,-21.213979,-8.215804,0
199,6.071158,-2.771887,10.972801,-6.992161,0
586,12.273573,8.164306,-12.881873,-11.386556,0


Evaluating Multiple Models

In [14]:
estimators = [2,5,10,20,30,50]
rfc_models = {}

for n_estimator in estimators:
    rfc = RandomForestClassifier(n_estimators=n_estimator)
    rfc.fit(X_train, y_train)
    rfc_models[str(n_estimator)] = rfc
    print(f"{rfc_models}")

{'2': RandomForestClassifier(n_estimators=2)}
{'2': RandomForestClassifier(n_estimators=2), '5': RandomForestClassifier(n_estimators=5)}
{'2': RandomForestClassifier(n_estimators=2), '5': RandomForestClassifier(n_estimators=5), '10': RandomForestClassifier(n_estimators=10)}
{'2': RandomForestClassifier(n_estimators=2), '5': RandomForestClassifier(n_estimators=5), '10': RandomForestClassifier(n_estimators=10), '20': RandomForestClassifier(n_estimators=20)}
{'2': RandomForestClassifier(n_estimators=2), '5': RandomForestClassifier(n_estimators=5), '10': RandomForestClassifier(n_estimators=10), '20': RandomForestClassifier(n_estimators=20), '30': RandomForestClassifier(n_estimators=30)}
{'2': RandomForestClassifier(n_estimators=2), '5': RandomForestClassifier(n_estimators=5), '10': RandomForestClassifier(n_estimators=10), '20': RandomForestClassifier(n_estimators=20), '30': RandomForestClassifier(n_estimators=30), '50': RandomForestClassifier(n_estimators=50)}


In [ ]:
estimators = [2, 5, 10, 20, 30, 50]
rfc_models = {}

for n_estimators in estimators:
    rfc = RandomForestClassifier(n_estimators=n_estimators)
    rfc.fit(X_train, y_train)
    rfc_models[f"{n_estimators}"] = rfc
    print(f"Trained {n_estimators} trees.")

with mlflow.start_run(run_name="evaluating-multiple-models") as parent_run:

    for n_estimators, model in rfc_models.items():
        with mlflow.start_run(
            run_name=f"rfc_{n_estimators}", 
            parent_run_id=parent_run.info.run_id, 
            nested=True
        ) as child_run:
            artifact_path = "random-forest-model"
            mlflow.sklearn.log_model(model, artifact_path)
            mlflow.log_param("n_estimators", int(n_estimators))

            result = mlflow.evaluate(
                model=f"runs:/{child_run.info.run_id}/{artifact_path}",
                data=eval_data,
                model_type="classifier",
                targets="target",
                feature_names=feature_names,
            )

Trained 2 trees.
Trained 5 trees.
Trained 10 trees.
Trained 20 trees.
Trained 30 trees.
Trained 50 trees.


2025/09/13 21:13:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
c:\Users\saqli\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/09/13 21:13:11 WARNING mlflow.models.evaluat